In [13]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [14]:
%cd /gdrive/MyDrive/Coursework/Fall\ 2021/682/Project/Dataset/preprocessed_images

/gdrive/MyDrive/Coursework/Fall 2021/682/Project/Dataset/preprocessed_images


In [15]:
!pip install git+https://github.com/rwightman/pytorch-image-models
!pip install albumentations -U

  Cloning https://github.com/rwightman/pytorch-image-models to /tmp/pip-req-build-00qhjr7l
  Running command git clone -q https://github.com/rwightman/pytorch-image-models /tmp/pip-req-build-00qhjr7l


In [16]:
import numpy as np 
import pandas as pd
import random
import os
import math
import time

In [17]:
from sklearn.metrics import accuracy_score
from sklearn.utils import class_weight
from PIL import Image as pil_image
from tqdm import tqdm
import scipy

import matplotlib
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

In [18]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

import timm
from timm.optim import Lookahead, RAdam

In [19]:
DATA_FOLDER = "/gdrive/MyDrive/Coursework/Fall 2021/682/Project/Dataset/preprocessed_images/"
print(len(os.listdir(DATA_FOLDER)))

5130


In [20]:
import albumentations as A
import albumentations.pytorch as APT
import cv2 

train_transform = A.Compose([
    # A.Resize(IMG_SIZE, IMG_SIZE),
    # A.CLAHE(p=1), 
    
    A.HorizontalFlip(p=0.75),
    A.VerticalFlip(p=0.25),
    A.ShiftScaleRotate(p=0.5, border_mode=cv2.BORDER_CONSTANT),
    A.OpticalDistortion(p=0.25),
    A.Perspective(p=0.25),
    A.CoarseDropout(p=0.5),

    A.RandomBrightness(p=0.75),
    A.ToFloat(),
    APT.transforms.ToTensorV2(),
])


valid_transform = A.Compose([
    # A.Resize(IMG_SIZE, IMG_SIZE),
    # A.CLAHE(p=1),
    A.ToFloat(),
    APT.transforms.ToTensorV2(),
])

/usr/local/lib/python3.7/dist-packages/albumentations/augmentations/transforms.py:1802: FutureWarning:

This class has been deprecated. Please use RandomBrightnessContrast



In [21]:
!pwd

/gdrive/My Drive/Coursework/Fall 2021/682/Project/Dataset/preprocessed_images


In [22]:
IMG_SIZE = 512
SEED = 42
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

In [23]:
# Load Train Data
class HotelTrainDataset:
    def __init__(self, data, transform=None, data_path="train_images/"):
        self.data = data
        self.data_path = data_path
        self.transform = transform
        self.fake_load = False

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        record = self.data.iloc[idx]
        image_path = self.data_path + record["image"]

        if self.fake_load:
            image = np.random.randint(0, 255, (32, 32, 3)).astype(np.uint8)
        else:
            image = np.array(pil_image.open(image_path)).astype(np.uint8)

        if self.transform:
            transformed = self.transform(image=image)
        
        return {
            "image" : transformed["image"],
            "target" : record['hotel_id_code'],
        }

In [24]:
class ArcMarginProduct(nn.Module):
    r"""Implement of large margin arc distance: :
        Args:
            in_features: size of each input sample
            out_features: size of each output sample
            s: norm of input feature
            m: margin
            cos(theta + m)
        """
    def __init__(self, in_features, out_features, s=30.0, m=0.50, easy_margin=False):
        super(ArcMarginProduct, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.s = s
        self.m = m
        self.weight = nn.Parameter(torch.FloatTensor(out_features, in_features))
        nn.init.xavier_uniform_(self.weight)

        self.easy_margin = easy_margin
        self.cos_m = math.cos(m)
        self.sin_m = math.sin(m)
        self.th = math.cos(math.pi - m)
        self.mm = math.sin(math.pi - m) * m

    def forward(self, input, label):
        # --------------------------- cos(theta) & phi(theta) ---------------------------
        cosine = F.linear(F.normalize(input), F.normalize(self.weight))
        sine = torch.sqrt((1.0 - torch.pow(cosine, 2)).clamp(0, 1))
        phi = cosine * self.cos_m - sine * self.sin_m
        if self.easy_margin:
            phi = torch.where(cosine > 0, phi, cosine)
        else:
            phi = torch.where(cosine > self.th, phi, cosine - self.mm)
        # --------------------------- convert label to one-hot ---------------------------
        # one_hot = torch.zeros(cosine.size(), requires_grad=True, device='cuda')
        one_hot = torch.zeros(cosine.size(), device='cuda')
        one_hot.scatter_(1, label.view(-1, 1).long(), 1)
        # -------------torch.where(out_i = {x_i if condition_i else y_i) -------------
        output = (one_hot * phi) + ((1.0 - one_hot) * cosine)  # you can use torch.where if your torch.__version__ is 0.4
        output *= self.s

        return output

class HotelIdModel(nn.Module):
    def __init__(self, out_features, embed_size=256, backbone_name="efficientnet_b3"):
        super(HotelIdModel, self).__init__()

        self.embed_size = embed_size
        self.backbone = timm.create_model(backbone_name, pretrained=True)
        in_features = self.backbone.get_classifier().in_features

        fc_name, _ = list(self.backbone.named_modules())[-1]
        if fc_name == 'classifier':
            self.backbone.classifier = nn.Identity()
        elif fc_name == 'head.fc':
            self.backbone.head.fc = nn.Identity()
        elif fc_name == 'fc':
            self.backbone.fc = nn.Identity()
        else:
            raise Exception("unknown classifier layer: " + fc_name)

        self.arc_face = ArcMarginProduct(self.embed_size, out_features, s=30.0, m=0.20, easy_margin=False)

        self.post = nn.Sequential(
            nn.utils.weight_norm(nn.Linear(in_features, self.embed_size*2), dim=None),
            nn.BatchNorm1d(self.embed_size*2),
            nn.Dropout(0.2),
            nn.utils.weight_norm(nn.Linear(self.embed_size*2, self.embed_size)),
            nn.BatchNorm1d(self.embed_size),
        )

        print(f"Model {backbone_name} ArcMarginProduct - Features: {in_features}, Embeds: {self.embed_size}")
        
    def forward(self, input, targets = None):
        x = self.backbone(input)
        x = x.view(x.size(0), -1)
        x = self.post(x)
        
        if targets is not None:
            logits = self.arc_face(x, targets)
            return logits
        
        return x

In [25]:
data_folder = "/gdrive/MyDrive/Coursework/Fall 2021/682/Project/Dataset/"
WIDTH = 512
HEIGHT = 512
train_df = pd.read_csv(f"{data_folder}train.csv" , parse_dates=["timestamp"])
data_df = train_df[train_df['chain'].isin([1,2])]
data_df["hotel_id_code"] = data_df["hotel_id"].astype('category').cat.codes.values.astype(np.int64)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [26]:
data_df.head()

,image,chain,hotel_id,timestamp,hotel_id_code
7,800e7dd55ffa4056.jpg,2,60181,2018-06-07 20:38:43,316
56,803dce35ca659b47.jpg,2,43373,2020-01-29 04:16:03,218
66,803ef0e73f60078f.jpg,2,63210,2019-05-28 20:33:56,329
79,803f7c814f2b2df4.jpg,2,60181,2018-08-01 12:05:06,316
87,803fd0d41f68c6d7.jpg,1,16966,2017-10-21 03:46:53,91


In [27]:
data_df["hotel_id_code"].nunique()

348

In [28]:
def train_epoch(args, model, loader, criterion, optimizer, scheduler, epoch):
    losses = []
    targets_all = []
    outputs_all = []
    
    model.train()
    t = tqdm(loader)
    
    for i, sample in enumerate(t):
        optimizer.zero_grad()
        
        input = sample['image'].to(args["device"])
        target = sample['target'].to(args["device"])
        
        output = model(input, target)
        loss = criterion(output, target)
        
        loss.backward()
        optimizer.step()

        if scheduler is not None:
            scheduler.step()
        
        losses.append(loss.item())
        targets_all.extend(target.cpu().numpy())
        outputs_all.extend(torch.sigmoid(output).detach().cpu().numpy())
        
        score = accuracy_score(targets_all, np.argmax(outputs_all, axis=1))
        e = args["epochs"]
        t.set_description(f"Epoch {epoch}/{e} - Train loss:{loss:0.4f}, score: {score:0.4f}")
        
    return np.mean(losses), score


In [43]:
def train_and_validate(args, data_df):
    # SEED and split
    seed_everything(seed=SEED)
    valid_df = data_df.groupby("hotel_id").sample(args["val_samples"], random_state=SEED)
    train_df = data_df[~data_df["image"].isin(valid_df["image"])]

    # create model
    model = HotelIdModel(args["n_classes"], args["embed_size"], args["backbone_name"])
    model = model.to(args["device"])

    # train data loader
    train_dataset = HotelTrainDataset(train_df, train_transform, data_path=DATA_FOLDER)
    train_loader = DataLoader(train_dataset, num_workers=args["num_workers"], batch_size=args["batch_size"], shuffle=True, pin_memory=False)
    # train without augmentations to generate base embeddings
    base_dataset = HotelTrainDataset(train_df, valid_transform, data_path=DATA_FOLDER)
    base_loader = DataLoader(base_dataset, num_workers=args["num_workers"], batch_size=args["batch_size"], shuffle=False)
    # valid loader
    valid_dataset = HotelTrainDataset(valid_df, valid_transform, data_path=DATA_FOLDER)
    valid_loader = DataLoader(valid_dataset, num_workers=args["num_workers"], batch_size=args["batch_size"], shuffle=False)

    print(f"Base: {len(base_dataset)}\nValidation: {len(valid_dataset)}")

    criterion = nn.CrossEntropyLoss()
    optimizer = Lookahead(torch.optim.AdamW(model.parameters(), lr=args["lr"]), k=3)
    scheduler = torch.optim.lr_scheduler.OneCycleLR(
                    optimizer,
                    max_lr=args["lr"],
                    epochs=args["epochs"],
                    steps_per_epoch=len(train_loader),
                    div_factor=10,
                    final_div_factor=1,
                    pct_start=0.1,
                    anneal_strategy="cos",
                )
    
    start_epoch = 1

    if args["continue_from_checkpoint"]:
        model, scheduler, optimizer, last_epoch = load_checkpoint(model, scheduler, optimizer, model_name)
        iterate_loader(train_loader, last_epoch)
        start_epoch = start_epoch + last_epoch

    torch.cuda.empty_cache()

    for epoch in range(start_epoch, args["epochs"] +1):
        train_loss, train_score = train_epoch(args, model, train_loader, criterion, optimizer, scheduler, epoch)
        #save_checkpoint(model, scheduler, optimizer, epoch, model_name, train_loss, train_score)
        #if (epoch == 1): #  or (epoch % 3) == 0:
        #    base_embeds, valid_embeds, valid_targets, val_preds, distance_matrix = test(base_loader, valid_loader, model)

    #base_embeds, valid_embeds, valid_targets, val_preds, distance_matrix = test(base_loader, valid_loader, model)

In [44]:
args = {
    "epochs" : 2,
    "lr" : 1e-3,
    "batch_size" : 8,
    "num_workers" : 2,
    "embed_size" : 4096,
    "val_samples" : 1,
    "backbone_name" : "efficientnet_b1",
    "n_classes" : data_df["hotel_id_code"].nunique(),
    "device" : ('cuda' if torch.cuda.is_available() else 'cpu'),
    "continue_from_checkpoint" : False
}

In [45]:
train_and_validate(args, data_df)

Model efficientnet_b1 ArcMarginProduct - Features: 1280, Embeds: 4096
Base: 4782
Validation: 348


Epoch 2/2 - Train loss:8.0493, score: 0.1050: 100%|██████████| 598/598 [07:45<00:00,  1.28it/s]


In [35]:
torch.cuda.empty_cache()